In [21]:
import numpy as np
import pandas as pd
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ShuffleSplit, RandomizedSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
test1 = pd.read_csv('../Data/Test1.csv', index_col=0)
test2 = pd.read_csv('../Data/Test2.csv', index_col=0)
test4 = pd.read_csv('../Data/Test4.csv', index_col=0)
data = pd.concat([test1, test2, test4])
data = data.sample(frac = 1) #shuffle 
data.head()

,a_enc_1,b_enc_1,c_enc_1,x_enc_1,y_enc_1,z_enc_1,a_enc_2,b_enc_2,c_enc_2,x_enc_2,y_enc_2,z_enc_2,fx_1,fy_1,fz_1,fx_2,fy_2,fz_2
t,,,,,,,,,,,,,,,,,,
1636594209,-89.996551,0.002879,0.005083,382.897232,270.801450,-106.388909,89.998117,0.000417,-179.999606,383.361181,274.682764,-101.746728,-136.137345,-1247.807494,2660.028005,-49.845970,727.452638,-657.739945
1636593134,-89.995815,0.002554,0.003515,317.105693,95.624695,-97.126959,89.999291,-0.000191,-179.999664,314.624113,89.221383,-90.691303,466.750111,1573.667018,2539.825879,-503.980366,-801.736002,-399.650118
1637020430,-89.996654,0.012281,0.004437,192.422193,766.378881,-112.734138,89.996354,-0.007972,179.997814,189.978330,765.275928,-110.283722,682.135045,210.042741,2462.849252,-567.456047,70.945299,-1047.283368
1637390013,-90.008724,-0.009463,0.009129,671.127934,1098.276762,-184.356997,89.993537,0.003356,179.995238,671.042039,1099.356503,-183.350810,397.722876,-364.640043,2018.214033,-5.775605,54.166172,-735.452033
1637014940,-89.998750,0.000534,0.001925,259.612728,826.268974,-69.990715,89.995634,0.000892,-179.997200,259.518079,830.251784,-65.366086,-133.383630,-1207.245382,2678.686215,-94.626310,750.313943,-620.727180


In [3]:
# train test split 
train = data.sample(frac = 0.8)
test = data.drop(train.index)

X_train = train.drop(['fx_1', 'fy_1', 'fz_1', 'fx_2', 'fy_2', 'fz_2'], axis=1)
y_train = train[['fx_1', 'fy_1', 'fz_1', 'fx_2', 'fy_2', 'fz_2']]
print(f"Training data shape X {X_train.shape} and y {y_train.shape}")

X_test = test.drop(['fx_1', 'fy_1', 'fz_1', 'fx_2', 'fy_2', 'fz_2'], axis=1)
y_test = test[['fx_1', 'fy_1', 'fz_1', 'fx_2', 'fy_2', 'fz_2']]
print(f"Testing data shape X {X_test.shape} and y {y_test.shape}")


Training data shape X (55391, 12) and y (55391, 6)
Testing data shape X (13848, 12) and y (13848, 6)


In [5]:
cv = ShuffleSplit(n_splits=5, test_size=.2, random_state=0)
model = LinearRegression()
cross_val_score(model, X_train, y_train, cv=cv)

array([0.59714627, 0.59062639, 0.60229568, 0.59751452, 0.59084583])

In [35]:
cv = ShuffleSplit(n_splits=5, test_size=.25, random_state=0)
model = KNeighborsRegressor()
cross_val_score(model, X_train, y_train, cv=cv)

array([0.94116684, 0.94065777, 0.93926709, 0.93909565, 0.94210869])

In [38]:
cv = ShuffleSplit(n_splits=5, test_size=.25, random_state=0)
model = RandomForestRegressor(max_depth=20, random_state=0)
cross_val_score(model, X_train, y_train, cv=cv)

array([0.97809509, 0.97722246, 0.97714546, 0.97624225, 0.97717037])

In [14]:
print("Test set RMSE:", mean_squared_error(y_test, rf_random.predict(X_test), squared=False))

Test set RMSE: 69.67882053661698


/usr/local/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [8]:
random_grid = {'bootstrap': [True, False],
               'max_depth': [10, 20, 30],
               'min_samples_leaf': [1, 2, 4],
               'min_samples_split': [2, 5, 10],
               'n_estimators': [130, 180, 230]}

rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=0, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=4, min_samples_split=5, n_estimators=230; total time= 4.8min
[CV] END bootstrap=True, max_depth=30, min_samples_leaf=4, min_samples_split=2, n_estimators=230; total time= 3.2min
[CV] END bootstrap=False, max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=230; total time= 3.3min
[CV] END bootstrap=False, max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=130; total time= 2.0min
[CV] END bootstrap=False, max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=130; total time= 2.3min
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=2, min_samples_split=5, n_estimators=130; total time= 3.5min
[CV] END bootstrap=True, max_depth=20, min_samples_leaf=4, min_samples_split=10, n_estimators=130; total time= 1.7min
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=4, min_samples_split=5, n_estimators=180; total time= 3

RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [130, 180, 230]},
                   random_state=0, verbose=2)

In [7]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [9]:
filename = 'RandomForestRegressor.pkl'
rf_random = pickle.load(open(filename, 'rb'))

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test)

Model Performance
Average Error: 19.7045 degrees.
Accuracy = 96.54%.


In [23]:
predictions = best_random.predict(X_test)
# Mean squared error 
MSE = mean_squared_error(y_test, predictions)

# Root mean square error 
rms = np.sqrt(mean_squared_error(y_test, predictions))

# R squared
r2 = r2_score(y_test, predictions) 
r2

0.994163048495325